## Applied Data Science Capstone Project

#### Importing the libraries

In [91]:
import requests 
import pandas as pd 
import numpy as np 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



#### Setting up Foursquare Data

In [92]:
ClIENT_ID = 'HWHQ5TRUCJGXTAYGMYLGNCTZTWCBNPIS51DBQ3UOLQRZEDRA' # your Foursquare ID
ClIENT_SECRET = 'ML4ERCN5FJUHA3NGQHFQX402HCSJJWXJ32PLLTGE2JS3IJIF' # your Foursquare Secret
VERSION = '20180505'
LIMIT =30
print('Your credentails:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentails:
Foursquare_ID: HWHQ5TRUCJGXTAYGMYLGNCTZTWCBNPIS51DBQ3UOLQRZEDRA
Foursquare_Secret:ML4ERCN5FJUHA3NGQHFQX402HCSJJWXJ32PLLTGE2JS3IJIF


#### City and Coordinates

In [93]:
city = 'New Delhi'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

28.6141793 77.2022662


#### Searching for Bookstores

In [94]:
search_query = 'Bookstore'
radius = 10000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=HWHQ5TRUCJGXTAYGMYLGNCTZTWCBNPIS51DBQ3UOLQRZEDRA&client_secret=ML4ERCN5FJUHA3NGQHFQX402HCSJJWXJ32PLLTGE2JS3IJIF&ll=28.6141793,77.2022662&v=20180505&query=Bookstore&radius=10000&limit=30'

In [95]:
results = requests.get(url).json()

In [96]:
venues = results['response']['venues']

dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d114951735', 'name': 'Bookstore', 'pluralName': 'Bookstores', 'shortName': 'Bookstore', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/bookstore_', 'suffix': '.png'}, 'primary': True}]",False,4bffbe2258af952141aaf1b6,"81 Block N, Outer Circle, Connaught Place",IN,New Delhi,India,Barakhamba Rd,2730,"[81 Block N, Outer Circle, Connaught Place (Barakhamba Rd), New Delhi 110001, Delhi, India]","[{'label': 'display', 'lat': 28.631342568553944, 'lng': 77.22222877968129}]",28.631343,77.222229,110001,Delhi,Oxford Bookstore,v-1577367695
1,"[{'id': '4bf58dd8d48988d114951735', 'name': 'Bookstore', 'pluralName': 'Bookstores', 'shortName': 'Bookstore', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/bookstore_', 'suffix': '.png'}, 'primary': True}]",False,4ce788eb678aa093731ee3ea,"1st & 2nd Flr., 23, Middle Lane, Khan Market",IN,New Delhi,India,NaN,2891,"[1st & 2nd Flr., 23, Middle Lane, Khan Market, New Delhi 110003, Delhi, India]","[{'label': 'display', 'lat': 28.600199673438794, 'lng': 77.22719772178468}]",28.600200,77.227198,110003,Delhi,Full Circle Bookstore,v-1577367695
2,"[{'id': '4bf58dd8d48988d1dc931735', 'name': 'Tea Room', 'pluralName': 'Tea Rooms', 'shortName': 'Tea Room', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/tearoom_', 'suffix': '.png'}, 'primary': True}]",False,51690773e4b09433491d990e,"81, N-Block, Outer Circle, Connaught Place",IN,New Delhi,India,NaN,2695,"[81, N-Block, Outer Circle, Connaught Place, New Delhi 110001, Delhi, India]","[{'label': 'display', 'lat': 28.630920473614825, 'lng': 77.2221936228799}]",28.630920,77.222194,110001,Delhi,Cha Bar | चा बार,v-1577367695
3,[],False,4e3d1b327d8b0e961064c873,NaN,IN,NaN,India,NaN,6689,[India],"[{'label': 'display', 'lat': 28.55439, 'lng': 77.19537}]",28.554390,77.195370,NaN,NaN,Midland bookstore,v-1577367695
4,"[{'id': '4bf58dd8d48988d114951735', 'name': 'Bookstore', 'pluralName': 'Bookstores', 'shortName': 'Bookstore', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/bookstore_', 'suffix': '.png'}, 'primary': True}]",False,4f59e523e4b0a5eaef6df08a,25,IN,New Delhi,India,Janpath Market,2270,"[25 (Janpath Market), New Delhi 110001, Delhi, India]","[{'label': 'display', 'lat': 28.627614699322084, 'lng': 77.21974259830216}]",28.627615,77.219743,110001,Delhi,Famous book store,v-1577367695


#### Cleaning the Bookstore Dataframe

In [97]:
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Oxford Bookstore,Bookstore,"81 Block N, Outer Circle, Connaught Place",IN,New Delhi,India,Barakhamba Rd,2730,"[81 Block N, Outer Circle, Connaught Place (Barakhamba Rd), New Delhi 110001, Delhi, India]","[{'label': 'display', 'lat': 28.631342568553944, 'lng': 77.22222877968129}]",28.631343,77.222229,110001,Delhi,4bffbe2258af952141aaf1b6
1,Full Circle Bookstore,Bookstore,"1st & 2nd Flr., 23, Middle Lane, Khan Market",IN,New Delhi,India,NaN,2891,"[1st & 2nd Flr., 23, Middle Lane, Khan Market, New Delhi 110003, Delhi, India]","[{'label': 'display', 'lat': 28.600199673438794, 'lng': 77.22719772178468}]",28.600200,77.227198,110003,Delhi,4ce788eb678aa093731ee3ea
2,Cha Bar | चा बार,Tea Room,"81, N-Block, Outer Circle, Connaught Place",IN,New Delhi,India,NaN,2695,"[81, N-Block, Outer Circle, Connaught Place, New Delhi 110001, Delhi, India]","[{'label': 'display', 'lat': 28.630920473614825, 'lng': 77.2221936228799}]",28.630920,77.222194,110001,Delhi,51690773e4b09433491d990e
3,Midland bookstore,None,NaN,IN,NaN,India,NaN,6689,[India],"[{'label': 'display', 'lat': 28.55439, 'lng': 77.19537}]",28.554390,77.195370,NaN,NaN,4e3d1b327d8b0e961064c873
4,Famous book store,Bookstore,25,IN,New Delhi,India,Janpath Market,2270,"[25 (Janpath Market), New Delhi 110001, Delhi, India]","[{'label': 'display', 'lat': 28.627614699322084, 'lng': 77.21974259830216}]",28.627615,77.219743,110001,Delhi,4f59e523e4b0a5eaef6df08a


In [98]:
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress', 'labeledLatLngs', 'state', 'id'], axis=1)
clean_dataframe2

,name,categories,address,lat,lng,postalCode
0,Oxford Bookstore,Bookstore,"81 Block N, Outer Circle, Connaught Place",28.631343,77.222229,110001
1,Full Circle Bookstore,Bookstore,"1st & 2nd Flr., 23, Middle Lane, Khan Market",28.600200,77.227198,110003
2,Cha Bar | चा बार,Tea Room,"81, N-Block, Outer Circle, Connaught Place",28.630920,77.222194,110001
3,Midland bookstore,None,NaN,28.554390,77.195370,NaN
4,Famous book store,Bookstore,25,28.627615,77.219743,110001
5,Lakshmi Book Store,Bookstore,18 Janpath Bhawan,28.629590,77.218506,110001
6,Bahrisons Bookstore,Bookstore,Dlf Saket,28.527848,77.206437,NaN
7,Half Price Bookstore,Bookstore,Select Citywalk,28.529231,77.219490,NaN
8,Wordsworth bookstore,Bookstore,NaN,28.630448,77.081716,NaN
9,Delhi Book Store,Bookstore,Darya Ganj,28.639880,77.250610,NaN


In [99]:
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3

,name,categories,address,lat,lng,postalCode
0,Oxford Bookstore,Bookstore,"81 Block N, Outer Circle, Connaught Place",28.631343,77.222229,110001
1,Full Circle Bookstore,Bookstore,"1st & 2nd Flr., 23, Middle Lane, Khan Market",28.600200,77.227198,110003
2,Cha Bar | चा बार,Tea Room,"81, N-Block, Outer Circle, Connaught Place",28.630920,77.222194,110001
4,Famous book store,Bookstore,25,28.627615,77.219743,110001
5,Lakshmi Book Store,Bookstore,18 Janpath Bhawan,28.629590,77.218506,110001


In [100]:
array= ['Bookstore']
book_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
book_dataframe

,name,categories,address,lat,lng,postalCode
0,Oxford Bookstore,Bookstore,"81 Block N, Outer Circle, Connaught Place",28.631343,77.222229,110001
1,Full Circle Bookstore,Bookstore,"1st & 2nd Flr., 23, Middle Lane, Khan Market",28.600200,77.227198,110003
4,Famous book store,Bookstore,25,28.627615,77.219743,110001
5,Lakshmi Book Store,Bookstore,18 Janpath Bhawan,28.629590,77.218506,110001


In [101]:
df_book = book_dataframe.drop_duplicates(subset='name', keep="first")
df_book

,name,categories,address,lat,lng,postalCode
0,Oxford Bookstore,Bookstore,"81 Block N, Outer Circle, Connaught Place",28.631343,77.222229,110001
1,Full Circle Bookstore,Bookstore,"1st & 2nd Flr., 23, Middle Lane, Khan Market",28.600200,77.227198,110003
4,Famous book store,Bookstore,25,28.627615,77.219743,110001
5,Lakshmi Book Store,Bookstore,18 Janpath Bhawan,28.629590,77.218506,110001


#### Searching for Cafes

In [102]:
search_query = 'Cafe'
radius = 10000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=HWHQ5TRUCJGXTAYGMYLGNCTZTWCBNPIS51DBQ3UOLQRZEDRA&client_secret=ML4ERCN5FJUHA3NGQHFQX402HCSJJWXJ32PLLTGE2JS3IJIF&ll=28.6141793,77.2022662&v=20180505&query=Cafe&radius=10000&limit=30'

In [103]:
cresults = requests.get(url).json()

In [104]:
venues = cresults['response']['venues']

caf_dataframe = json_normalize(venues)
caf_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d110941735', 'name': 'Italian Restaurant', 'pluralName': 'Italian Restaurants', 'shortName': 'Italian', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/italian_', 'suffix': '.png'}, 'primary': True}]",False,4b701cbaf964a5209b072de3,68A,IN,New Delhi,India,Khan Market,2898,"[68A (Khan Market), New Delhi 110048, Delhi, India]","[{'label': 'display', 'lat': 28.60068629868818, 'lng': 77.22763621585838}]",28.600686,77.227636,NaN,110048,Delhi,The Big Chill Cafe,v-1577367784
1,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'Coffee Shop', 'pluralName': 'Coffee Shops', 'shortName': 'Coffee Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_', 'suffix': '.png'}, 'primary': True}]",False,50bcb63ee4b0bedc1d3c28f7,C Block,IN,Delhi,India,CP,2852,"[C Block (CP), Delhi, Delhi, India]","[{'label': 'display', 'lat': 28.6342949764577, 'lng': 77.22034493677113}]",28.634295,77.220345,NaN,NaN,Delhi,Cafe Coffee Day Square,v-1577367784
2,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'Coffee Shop', 'pluralName': 'Coffee Shops', 'shortName': 'Coffee Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_', 'suffix': '.png'}, 'primary': True}]",False,5117d099e4b0bae1afcea5c6,Inside Regal Building,IN,New Delhi,India,"Connaught Place, Near Regal Cinema",2365,"[Inside Regal Building (Connaught Place, Near Regal Cinema), New Delhi 110001, Delhi, India]","[{'label': 'display', 'lat': 28.631054, 'lng': 77.21697834826568}]",28.631054,77.216978,NaN,110001,Delhi,Café Coffee Day,v-1577367784
3,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'Coffee Shop', 'pluralName': 'Coffee Shops', 'shortName': 'Coffee Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_', 'suffix': '.png'}, 'primary': True}]",False,5279c495498e3592ada19181,NaN,IN,NaN,India,NaN,2414,[India],"[{'label': 'display', 'lat': 28.63095845041277, 'lng': 77.21792927048423}]",28.630958,77.217929,NaN,NaN,NaN,Cafe Coffee Day,v-1577367784
4,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'Café', 'pluralName': 'Cafés', 'shortName': 'Café', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_', 'suffix': '.png'}, 'primary': True}]",False,4d6b798b2ea9b1f78cf9d028,Outer Circle Connaught Place,IN,New Delhi,India,N Blk,2695,"[Outer Circle Connaught Place (N Blk), New Delhi, Delhi, India]","[{'label': 'display', 'lat': 28.63133963244659, 'lng': 77.22172567577032}]",28.631340,77.221726,NaN,NaN,Delhi,Cafe Coffee Day,v-1577367784


#### Cleaning the Cafes Dataframe

In [105]:
caf_clean_columns = ['name', 'categories'] + [col for col in caf_dataframe.columns if col.startswith('location.')]+ ['id']
clean_caf_dataframe = caf_dataframe.loc[:,caf_clean_columns]

def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

clean_caf_dataframe['categories'] = clean_caf_dataframe.apply(get_category_type, axis=1)

clean_caf_dataframe.columns = [column.split('.')[-1] for column in clean_caf_dataframe.columns]

clean_caf_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,The Big Chill Cafe,Italian Restaurant,68A,IN,New Delhi,India,Khan Market,2898,"[68A (Khan Market), New Delhi 110048, Delhi, India]","[{'label': 'display', 'lat': 28.60068629868818, 'lng': 77.22763621585838}]",28.600686,77.227636,NaN,110048,Delhi,4b701cbaf964a5209b072de3
1,Cafe Coffee Day Square,Coffee Shop,C Block,IN,Delhi,India,CP,2852,"[C Block (CP), Delhi, Delhi, India]","[{'label': 'display', 'lat': 28.6342949764577, 'lng': 77.22034493677113}]",28.634295,77.220345,NaN,NaN,Delhi,50bcb63ee4b0bedc1d3c28f7
2,Café Coffee Day,Coffee Shop,Inside Regal Building,IN,New Delhi,India,"Connaught Place, Near Regal Cinema",2365,"[Inside Regal Building (Connaught Place, Near Regal Cinema), New Delhi 110001, Delhi, India]","[{'label': 'display', 'lat': 28.631054, 'lng': 77.21697834826568}]",28.631054,77.216978,NaN,110001,Delhi,5117d099e4b0bae1afcea5c6
3,Cafe Coffee Day,Coffee Shop,NaN,IN,NaN,India,NaN,2414,[India],"[{'label': 'display', 'lat': 28.63095845041277, 'lng': 77.21792927048423}]",28.630958,77.217929,NaN,NaN,NaN,5279c495498e3592ada19181
4,Cafe Coffee Day,Café,Outer Circle Connaught Place,IN,New Delhi,India,N Blk,2695,"[Outer Circle Connaught Place (N Blk), New Delhi, Delhi, India]","[{'label': 'display', 'lat': 28.63133963244659, 'lng': 77.22172567577032}]",28.631340,77.221726,NaN,NaN,Delhi,4d6b798b2ea9b1f78cf9d028


In [106]:
clean_caf_dataframe2= clean_caf_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'state', 'neighborhood', 'postalCode', 'id'], axis=1)
clean_caf_dataframe2

,name,categories,address,lat,lng
0,The Big Chill Cafe,Italian Restaurant,68A,28.600686,77.227636
1,Cafe Coffee Day Square,Coffee Shop,C Block,28.634295,77.220345
2,Café Coffee Day,Coffee Shop,Inside Regal Building,28.631054,77.216978
3,Cafe Coffee Day,Coffee Shop,NaN,28.630958,77.217929
4,Cafe Coffee Day,Café,Outer Circle Connaught Place,28.631340,77.221726
5,Cafe,Café,CMYK,28.586478,77.226498
6,Cafe Coffee Day,Café,"Pallavi AWWA Complex, Chanakyapuri",28.595247,77.171954
7,Mysore Cafe,Tea Room,NaN,28.608261,77.201475
8,Cafe Coffee Day,Café,Connaught Place,28.633841,77.217909
9,The Beer Cafe,Beer Garden,Scindia House,28.629139,77.219603


In [107]:
clean_caf_dataframe3 = clean_caf_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_caf_dataframe3

,name,categories,address,lat,lng
0,The Big Chill Cafe,Italian Restaurant,68A,28.600686,77.227636
1,Cafe Coffee Day Square,Coffee Shop,C Block,28.634295,77.220345
2,Café Coffee Day,Coffee Shop,Inside Regal Building,28.631054,77.216978
4,Cafe Coffee Day,Café,Outer Circle Connaught Place,28.631340,77.221726
5,Cafe,Café,CMYK,28.586478,77.226498
6,Cafe Coffee Day,Café,"Pallavi AWWA Complex, Chanakyapuri",28.595247,77.171954
8,Cafe Coffee Day,Café,Connaught Place,28.633841,77.217909
9,The Beer Cafe,Beer Garden,Scindia House,28.629139,77.219603
10,Café Nirvana,Café,Main Bazar Rd.,28.641334,77.214683
11,Boombox Cafe,Lounge,"Rajiv Chowk, F Block",28.631275,77.220041


In [108]:
df_caf = clean_caf_dataframe3[clean_caf_dataframe3.categories == 'Café']
df_caf

,name,categories,address,lat,lng
4,Cafe Coffee Day,Café,Outer Circle Connaught Place,28.631340,77.221726
5,Cafe,Café,CMYK,28.586478,77.226498
6,Cafe Coffee Day,Café,"Pallavi AWWA Complex, Chanakyapuri",28.595247,77.171954
8,Cafe Coffee Day,Café,Connaught Place,28.633841,77.217909
10,Café Nirvana,Café,Main Bazar Rd.,28.641334,77.214683
12,1 Cafe Lounge,Café,"Outer Circle, Connaught Place",28.633142,77.222518
20,Triveni Terrace Cafe,Café,Triveni Kala Sangam,28.627295,77.232677
22,Cafe Coffee Day,Café,Delhi Airport Metro Express,28.617903,77.188474
23,The Pool Cafe @ The Lodhi Hotel,Café,The Lodhi Hotel,28.591559,77.238278
24,Cafe Delhi Heights,Café,"68/1, Janpath, Atul Grove Road, Janpath, Connaught Place, New Delhi, Delhi 110001",28.627842,77.219860


#### Searching for Theatres

In [109]:
search_query = 'Theatre'
radius = 10000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=HWHQ5TRUCJGXTAYGMYLGNCTZTWCBNPIS51DBQ3UOLQRZEDRA&client_secret=ML4ERCN5FJUHA3NGQHFQX402HCSJJWXJ32PLLTGE2JS3IJIF&ll=28.6141793,77.2022662&v=20180505&query=Theatre&radius=10000&limit=30'

In [110]:
tresults = requests.get(url).json()

In [111]:
venues = tresults['response']['venues']

th_dataframe = json_normalize(venues)
th_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d137941735', 'name': 'Theater', 'pluralName': 'Theaters', 'shortName': 'Theater', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/performingarts_theater_', 'suffix': '.png'}, 'primary': True}]",False,4d0a11f8e6f83704ef085e87,"11-B, Baba Kharak Singh Marg",IN,New Delhi,India,Next to RML Hospital exit no. 5,1283,"[11-B, Baba Kharak Singh Marg (Next to RML Hospital exit no. 5), New Delhi 110001, Delhi, India]","[{'label': 'display', 'lat': 28.62566512551797, 'lng': 77.20337900510125}]",28.625665,77.203379,110001,Delhi,Akshara Theatre | अक्षरा थिएटर,v-1577367792,NaN
1,"[{'id': '5032792091d4c4b30a586d5c', 'name': 'Concert Hall', 'pluralName': 'Concert Halls', 'shortName': 'Concert Hall', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/musicvenue_', 'suffix': '.png'}, 'primary': True}]",False,516174b1e4b0878cfbc6f827,NaN,IN,NaN,India,NaN,1529,[India],"[{'label': 'display', 'lat': 28.625151, 'lng': 77.19284}]",28.625151,77.192840,NaN,NaN,Surtal(Open Air Theatre) Talkatora,v-1577367792,NaN
2,"[{'id': '4bf58dd8d48988d137941735', 'name': 'Theater', 'pluralName': 'Theaters', 'shortName': 'Theater', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/performingarts_theater_', 'suffix': '.png'}, 'primary': True}]",False,507ab579e4b0753d5a388f68,NaN,IN,NaN,India,NaN,2929,[India],"[{'label': 'display', 'lat': 28.614031271616838, 'lng': 77.23224425883865}]",28.614031,77.232244,NaN,NaN,Meghdoot Theatre,v-1577367792,NaN
3,"[{'id': '4bf58dd8d48988d137941735', 'name': 'Theater', 'pluralName': 'Theaters', 'shortName': 'Theater', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/performingarts_theater_', 'suffix': '.png'}, 'primary': True}]",False,5098fb675262fd71fdd1564c,COPERNICUS MARG,IN,Delhi,India,DOORDARSHAN BHAWAN,2990,"[COPERNICUS MARG (DOORDARSHAN BHAWAN), Delhi 110001, Delhi, India]","[{'label': 'display', 'lat': 28.6218039088934, 'lng': 77.23161220550537}]",28.621804,77.231612,110001,Delhi,little theatre group,v-1577367792,NaN
4,"[{'id': '4bf58dd8d48988d137941735', 'name': 'Theater', 'pluralName': 'Theaters', 'shortName': 'Theater', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/performingarts_theater_', 'suffix': '.png'}, 'primary': True}]",False,4f4dfec0e4b088c4aed32436,Nav shakti school,IN,NaN,India,ITO,4320,"[Nav shakti school (ITO), India]","[{'label': 'display', 'lat': 28.632810337539144, 'lng': 77.24105132416719}]",28.632810,77.241051,NaN,NaN,asmita theatre,v-1577367792,NaN


#### Cleaning the Theatres Dataframe

In [112]:
th_clean_columns = ['name', 'categories'] + [col for col in th_dataframe.columns if col.startswith('location.')]+ ['id']
clean_th_dataframe = th_dataframe.loc[:,th_clean_columns]

def get_category_type(row):
    try:
        categories_list3 = row['categories']
    except:
        categories_list3 = row['venue.categories']
        
    if len(categories_list3) == 0:
        return None
    else:
        return categories_list3[0]['name']

clean_th_dataframe['categories'] = clean_th_dataframe.apply(get_category_type, axis=1)

clean_th_dataframe.columns = [column.split('.')[-1] for column in clean_th_dataframe.columns]

clean_th_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Akshara Theatre | अक्षरा थिएटर,Theater,"11-B, Baba Kharak Singh Marg",IN,New Delhi,India,Next to RML Hospital exit no. 5,1283,"[11-B, Baba Kharak Singh Marg (Next to RML Hospital exit no. 5), New Delhi 110001, Delhi, India]","[{'label': 'display', 'lat': 28.62566512551797, 'lng': 77.20337900510125}]",28.625665,77.203379,110001,Delhi,4d0a11f8e6f83704ef085e87
1,Surtal(Open Air Theatre) Talkatora,Concert Hall,NaN,IN,NaN,India,NaN,1529,[India],"[{'label': 'display', 'lat': 28.625151, 'lng': 77.19284}]",28.625151,77.192840,NaN,NaN,516174b1e4b0878cfbc6f827
2,Meghdoot Theatre,Theater,NaN,IN,NaN,India,NaN,2929,[India],"[{'label': 'display', 'lat': 28.614031271616838, 'lng': 77.23224425883865}]",28.614031,77.232244,NaN,NaN,507ab579e4b0753d5a388f68
3,little theatre group,Theater,COPERNICUS MARG,IN,Delhi,India,DOORDARSHAN BHAWAN,2990,"[COPERNICUS MARG (DOORDARSHAN BHAWAN), Delhi 110001, Delhi, India]","[{'label': 'display', 'lat': 28.6218039088934, 'lng': 77.23161220550537}]",28.621804,77.231612,110001,Delhi,5098fb675262fd71fdd1564c
4,asmita theatre,Theater,Nav shakti school,IN,NaN,India,ITO,4320,"[Nav shakti school (ITO), India]","[{'label': 'display', 'lat': 28.632810337539144, 'lng': 77.24105132416719}]",28.632810,77.241051,NaN,NaN,4f4dfec0e4b088c4aed32436


In [113]:
clean_th_dataframe2= clean_th_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'postalCode', 'state', 'id'], axis=1)
clean_th_dataframe2

,name,categories,address,lat,lng
0,Akshara Theatre | अक्षरा थिएटर,Theater,"11-B, Baba Kharak Singh Marg",28.625665,77.203379
1,Surtal(Open Air Theatre) Talkatora,Concert Hall,NaN,28.625151,77.192840
2,Meghdoot Theatre,Theater,NaN,28.614031,77.232244
3,little theatre group,Theater,COPERNICUS MARG,28.621804,77.231612
4,asmita theatre,Theater,Nav shakti school,28.632810,77.241051
5,"Surgery Operation Theatre, LNJP Hospital",College Academic Building,NaN,28.638393,77.236830
6,Open Air Theatre,Indie Movie Theater,Taurus,28.598426,77.141919
7,Actor Factor Theatre Company,Theater,NaN,28.548530,77.211810
8,The hoshruba Theatre Company,Theater,209 A Peacock Lane,28.547016,77.213792
9,ARIA Music & Theatre Conservatory,Music Venue,C-52 A,28.646652,77.122512


In [114]:
df_th = clean_th_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
df_th

,name,categories,address,lat,lng
0,Akshara Theatre | अक्षरा थिएटर,Theater,"11-B, Baba Kharak Singh Marg",28.625665,77.203379
3,little theatre group,Theater,COPERNICUS MARG,28.621804,77.231612
4,asmita theatre,Theater,Nav shakti school,28.632810,77.241051
6,Open Air Theatre,Indie Movie Theater,Taurus,28.598426,77.141919
8,The hoshruba Theatre Company,Theater,209 A Peacock Lane,28.547016,77.213792
9,ARIA Music & Theatre Conservatory,Music Venue,C-52 A,28.646652,77.122512
10,Open Air Theatre,College Theater,JNU UNIVERSITY,28.548194,77.168010
12,Oddbird Theatre,Indie Theater,"Dhan Mill Compound, 100 Foot Road, SSN Marg, Pocket D, Dr Ambedkar Colony, Chhattarpur, New Delhi, Delhi 110030",28.506228,77.183645
13,The Atre Yoga Studio,Yoga Studio,"252-A, Nanak Bhavan",28.548267,77.211383


#### Searching for Parks

In [115]:
search_query = 'Park'
radius = 10000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=HWHQ5TRUCJGXTAYGMYLGNCTZTWCBNPIS51DBQ3UOLQRZEDRA&client_secret=ML4ERCN5FJUHA3NGQHFQX402HCSJJWXJ32PLLTGE2JS3IJIF&ll=28.6141793,77.2022662&v=20180505&query=Park&radius=10000&limit=30'

In [116]:
presults = requests.get(url).json()

In [117]:
venues = presults['response']['venues']

park_dataframe = json_normalize(venues)
park_dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'Hotel', 'pluralName': 'Hotels', 'shortName': 'Hotel', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/hotel_', 'suffix': '.png'}, 'primary': True}]",False,4b80e458f964a520bf9030e3,15 Parliament St,IN,New Delhi,India,NaN,2135,"[15 Parliament St, New Delhi 110 001, Delhi, India]","[{'label': 'display', 'lat': 28.628731170539613, 'lng': 77.21650745045316}]",28.628731,77.216507,NaN,110 001,Delhi,The Park Hotel,v-1577367803
1,"[{'id': '4bf58dd8d48988d163941735', 'name': 'Park', 'pluralName': 'Parks', 'shortName': 'Park', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_', 'suffix': '.png'}, 'primary': True}]",False,5bd99a5d8ad62e002cc35a90,NaN,IN,New Delhi,India,NaN,2682,"[New Delhi 110001, Delhi, India]","[{'label': 'display', 'lat': 28.632954, 'lng': 77.21947}]",28.632954,77.219470,NaN,110001,Delhi,Central Park - New Delhi,v-1577367803
2,"[{'id': '4bf58dd8d48988d163941735', 'name': 'Park', 'pluralName': 'Parks', 'shortName': 'Park', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_', 'suffix': '.png'}, 'primary': True}]",False,4ce89bd9e888f04dbda7436b,Chanakyapuri,IN,New Delhi,India,NaN,2655,"[Chanakyapuri, New Delhi, Delhi, India]","[{'label': 'display', 'lat': 28.591798206029363, 'lng': 77.19285983211086}]",28.591798,77.192860,NaN,NaN,Delhi,Nehru Park | नेहरू पार्क (Nehru Park),v-1577367803
3,"[{'id': '4bf58dd8d48988d163941735', 'name': 'Park', 'pluralName': 'Parks', 'shortName': 'Park', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_', 'suffix': '.png'}, 'primary': True}]",False,4e18a5cd81309771577a1f9a,Pandara Park,IN,New Delhi,India,Lodi Estate,3100,"[Pandara Park (Lodi Estate), New Delhi 110003, Delhi, India]","[{'label': 'display', 'lat': 28.60352005138293, 'lng': 77.23158001899719}]",28.603520,77.231580,NaN,110003,Delhi,Pandara Rd Local Park,v-1577367803
4,"[{'id': '4bf58dd8d48988d163941735', 'name': 'Park', 'pluralName': 'Parks', 'shortName': 'Park', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_', 'suffix': '.png'}, 'primary': True}]",False,5dac8f1928f17a000861f133,NaN,IN,ニューデリー,India,NaN,2122,"[ニューデリー 110001, デリー, India]","[{'label': 'display', 'lat': 28.629497, 'lng': 77.215203}]",28.629497,77.215203,NaN,110001,デリー,Pracheen Hanuman Mandir Park,v-1577367803


#### Cleaning the Parks Dataframe

In [118]:
park_clean_columns = ['name', 'categories'] + [col for col in park_dataframe.columns if col.startswith('location.')]+ ['id']
clean_park_dataframe = park_dataframe.loc[:,park_clean_columns]

def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

clean_park_dataframe['categories'] = clean_park_dataframe.apply(get_category_type, axis=1)

clean_park_dataframe.columns = [column.split('.')[-1] for column in clean_park_dataframe.columns]

clean_park_dataframe.head()

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,The Park Hotel,Hotel,15 Parliament St,IN,New Delhi,India,NaN,2135,"[15 Parliament St, New Delhi 110 001, Delhi, India]","[{'label': 'display', 'lat': 28.628731170539613, 'lng': 77.21650745045316}]",28.628731,77.216507,NaN,110 001,Delhi,4b80e458f964a520bf9030e3
1,Central Park - New Delhi,Park,NaN,IN,New Delhi,India,NaN,2682,"[New Delhi 110001, Delhi, India]","[{'label': 'display', 'lat': 28.632954, 'lng': 77.21947}]",28.632954,77.219470,NaN,110001,Delhi,5bd99a5d8ad62e002cc35a90
2,Nehru Park | नेहरू पार्क (Nehru Park),Park,Chanakyapuri,IN,New Delhi,India,NaN,2655,"[Chanakyapuri, New Delhi, Delhi, India]","[{'label': 'display', 'lat': 28.591798206029363, 'lng': 77.19285983211086}]",28.591798,77.192860,NaN,NaN,Delhi,4ce89bd9e888f04dbda7436b
3,Pandara Rd Local Park,Park,Pandara Park,IN,New Delhi,India,Lodi Estate,3100,"[Pandara Park (Lodi Estate), New Delhi 110003, Delhi, India]","[{'label': 'display', 'lat': 28.60352005138293, 'lng': 77.23158001899719}]",28.603520,77.231580,NaN,110003,Delhi,4e18a5cd81309771577a1f9a
4,Pracheen Hanuman Mandir Park,Park,NaN,IN,ニューデリー,India,NaN,2122,"[ニューデリー 110001, デリー, India]","[{'label': 'display', 'lat': 28.629497, 'lng': 77.215203}]",28.629497,77.215203,NaN,110001,デリー,5dac8f1928f17a000861f133


In [119]:
clean_park_dataframe2= clean_park_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'neighborhood', 'postalCode', 'state', 'id'], axis=1)
clean_park_dataframe2

,name,categories,address,lat,lng
0,The Park Hotel,Hotel,15 Parliament St,28.628731,77.216507
1,Central Park - New Delhi,Park,NaN,28.632954,77.219470
2,Nehru Park | नेहरू पार्क (Nehru Park),Park,Chanakyapuri,28.591798,77.192860
3,Pandara Rd Local Park,Park,Pandara Park,28.603520,77.231580
4,Pracheen Hanuman Mandir Park,Park,NaN,28.629497,77.215203
5,National Zoological Park | राष्ट्रीय प्राणी उद्यान,Zoo,Off Sunder Nagar,28.607453,77.241686
6,Triangle Park,Park,Pandara Road,28.604606,77.228823
7,Deer Park,Park,Safdarjung Enclave,28.558191,77.193814
8,Gulmohar Park Road,Park,177 Adhar Shila,28.559251,77.213404
9,Park Inn by Radisson New Delhi Lajpat Nagar,Hotel,"1&2, Ring Road, Lajpat Nagar 4,",28.563750,77.234810


In [120]:
clean_park_dataframe3 = clean_park_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_park_dataframe3

,name,categories,address,lat,lng
0,The Park Hotel,Hotel,15 Parliament St,28.628731,77.216507
2,Nehru Park | नेहरू पार्क (Nehru Park),Park,Chanakyapuri,28.591798,77.192860
3,Pandara Rd Local Park,Park,Pandara Park,28.603520,77.231580
5,National Zoological Park | राष्ट्रीय प्राणी उद्यान,Zoo,Off Sunder Nagar,28.607453,77.241686
6,Triangle Park,Park,Pandara Road,28.604606,77.228823
7,Deer Park,Park,Safdarjung Enclave,28.558191,77.193814
8,Gulmohar Park Road,Park,177 Adhar Shila,28.559251,77.213404
9,Park Inn by Radisson New Delhi Lajpat Nagar,Hotel,"1&2, Ring Road, Lajpat Nagar 4,",28.563750,77.234810
10,Hotel Kingston Park New Delhi,Hotel,8/5 WEA Karol Bagh,28.645230,77.192467
11,Panchsheel Park | पंचशील पार्क,Park,Josip Broz Tito Marg,28.553746,77.228652


In [121]:
df_park = clean_park_dataframe3[clean_park_dataframe3.categories == 'Park']
df_park

,name,categories,address,lat,lng
2,Nehru Park | नेहरू पार्क (Nehru Park),Park,Chanakyapuri,28.591798,77.192860
3,Pandara Rd Local Park,Park,Pandara Park,28.603520,77.231580
6,Triangle Park,Park,Pandara Road,28.604606,77.228823
7,Deer Park,Park,Safdarjung Enclave,28.558191,77.193814
8,Gulmohar Park Road,Park,177 Adhar Shila,28.559251,77.213404
11,Panchsheel Park | पंचशील पार्क,Park,Josip Broz Tito Marg,28.553746,77.228652
14,Thyagraj Park,Park,Thyagraj Park,28.578247,77.214983
18,Central Park,Park,Connaught Place,28.632614,77.219299


## Generate map to visualize Bookstores, Cafes, Theatres and Parks to see how they Cluster

In [122]:
proj_neighbourhood_df = pd.concat([df_park, df_book, df_caf, df_th], sort=True)
proj_neighbourhood_df

,address,categories,lat,lng,name,postalCode
2,Chanakyapuri,Park,28.591798,77.192860,Nehru Park | नेहरू पार्क (Nehru Park),NaN
3,Pandara Park,Park,28.603520,77.231580,Pandara Rd Local Park,NaN
6,Pandara Road,Park,28.604606,77.228823,Triangle Park,NaN
7,Safdarjung Enclave,Park,28.558191,77.193814,Deer Park,NaN
8,177 Adhar Shila,Park,28.559251,77.213404,Gulmohar Park Road,NaN
11,Josip Broz Tito Marg,Park,28.553746,77.228652,Panchsheel Park | पंचशील पार्क,NaN
14,Thyagraj Park,Park,28.578247,77.214983,Thyagraj Park,NaN
18,Connaught Place,Park,28.632614,77.219299,Central Park,NaN
0,"81 Block N, Outer Circle, Connaught Place",Bookstore,28.631343,77.222229,Oxford Bookstore,110001
1,"1st & 2nd Flr., 23, Middle Lane, Khan Market",Bookstore,28.600200,77.227198,Full Circle Bookstore,110003


In [123]:
proj_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(proj_neighbourhood_df['lat'], proj_neighbourhood_df['lng'], 
                                           proj_neighbourhood_df['name'], proj_neighbourhood_df['categories'],\
                                               proj_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(proj_map)  
    
proj_map